In [1]:
import pymongo as pym
import pandas as pd
import requests
import nltk
from bs4 import BeautifulSoup
#from bs4 import BeautifulSoup as bs
import urllib.request, urllib.parse, urllib.error
from newspaper import Article 
import schedule
import time

In [6]:
def rss_feed_schedule():
    url=["https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml","https://blockchain.news/RSS/","https://news.bitcoin.com/feed/"]
    #Making Connection with mongodb
    client = pym.MongoClient('mongodb://localhost:27017/')
        #creating db
    db = client["Rss_Feed"]
        #creating tables
    rss_feeds_table = db["Rss_Feed_Info"]
    for u in url:
      

        resp=requests.get(u)
        #soup = bs(resp.text, 'html.parser')
        soup=BeautifulSoup(resp.content,features='xml')
        news = []
        items=soup.findAll('item')
        #nltk.download('punkt')
        item = iter(items)
        for item in items:
            news_item={}
            #"PubDate":item.pubDate.text,
            #Check if there is document with PubDate and Title specified
            if db.Rss_Feed_Info.find_one({"Title":item.title.text}) or db.Rss_Feed_Info.find_one({"Title":None}):
                if db.Rss_Feed_Info.find_one({"PubDate":item.pubDate.text}) or db.Rss_Feed_Info.find_one({"PubDate":None}):
                    #print("Data present")
                    #If The document with specified PubDate and Title is present the news_item store null values
                    news_item['Title']=None
                    news_item['Link']=None
                    news_item['Description']=None
                    news_item['PubDate']=None
                    #news_item['Category']=None
                    news_item['Keyword']=None
                    #next(item)
            else:  
                #If The document with specified PubDate and Title is not present news_item stores the values
                news_item['Title']=item.title.text
                news_item['Link']=item.link.text
                news_item['Description']=item.description.text
                news_item['PubDate']=item.pubDate.text
                #news_item['Category']=item.category.text
                a = item.link.text
                article = Article(a)
                article.download()
                article.parse() 
                article.nlp()
                news_item['Keyword']=article.keywords
            news.append(news_item)

        df = pd.DataFrame(news)
        #df.head(26)
        

        # Create Dict() to send to MongoDB
        rss_feed_dict = df.to_dict('record')
        rss_feeds_table.insert_many(rss_feed_dict)

        #Removing Null Values
        db.Rss_Feed_Info.delete_many({ "Title": None , "Link" : None , "Description": None, "PubDate": None})
    #db.Rss_Feed_Info.find({},{"PubDate":-1}).sort("PubDate",-1)
        

#Scheduling every 1 hr 
schedule.every(4).seconds.do(rss_feed_schedule)
#schedule.every().hour.do(rss_feed_schedule)
while True:
    schedule.run_pending()
    #print("Data")
    time.sleep(1)
    #print("Added")


'#Scheduling every 1 hr \nschedule.every(4).seconds.do(rss_feed_schedule)\n#schedule.every().hour.do(rss_feed_schedule)\nwhile True:\n    schedule.run_pending()\n    #print("Data")\n    time.sleep(1)\n    #print("Added")'

In [ ]:
df.head(25)

In [7]:
rss_feed_schedule()
